# Fit a manoeuvring model on raw data and filtered data

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.models.regression import MotionRegression, Regression

from vessel_manoeuvring_models.parameters import df_parameters
from vessel_manoeuvring_models.substitute_dynamic_symbols import run
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from vessel_manoeuvring_models.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from vessel_manoeuvring_models.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter, extended_kalman_smoother

In [ ]:
%reload_kedro

#id = "MARIN_FREE_KVLCC2_tc_35_m"
id = "MARIN_FREE_KVLCC2_zz_20_m"
#id = "MARIN_FREE_KVLCC2_tc_35_m"

df_raw = catalog.load(f"kvlcc2.{id}.raw_data")
data = catalog.load(f"kvlcc2.{id}.data")
df_smooth = catalog.load(f"kvlcc2.initial.{id}.data_ek_smooth")
df_ek = catalog.load(f"kvlcc2.initial.{id}.data_ek_filter")


#df_raw.index = df_raw.index-4
dataframes = {'raw':data,
              'df_ek': df_ek,
              'smooth' : df_smooth,        
             }

styles = {

    'df_ek':{'alpha':0.5},
}

ek = catalog.load("kvlcc2.vmm_abkowitz.ek")
covariance_matrixes = catalog.load("kvlcc2.vmm_martin.covariance_matrixes")
x0 = catalog.load(f"kvlcc2.initial.{id}.x0")
parameters = catalog.load(f"kvlcc2.initial.vmm_martin.joined.derivatives")

In [ ]:
def filter_with_updated_model(ek, parameters, data, covariance_matrixes, x0):
    
    ek2 = ek.copy()
    ek2.parameters.update(parameters['regressed'])
    ek_filtered, data_ek_filter, time_steps = extended_kalman_filter(ek=ek2, data=data, covariance_matrixes=covariance_matrixes, x0=x0, hydrodynamic_derivatives=parameters)
    ek_smooth, data_ek_smooth = extended_kalman_smoother(ek=ek_filtered, data=data, time_steps=time_steps, covariance_matrixes=covariance_matrixes, hydrodynamic_derivatives=parameters)
    return data_ek_smooth

In [ ]:
dataframes['smooth2'] = filter_with_updated_model(ek, parameters, data, covariance_matrixes, x0)

In [ ]:
plot(dataframes=dataframes, keys=["thrust", "psi", "u", "v", "r", "u1d","v1d","r1d"], styles=styles);

In [ ]:
%reload_kedro
added_masses = catalog.load("kvlcc2.added_masses")
ship_data = catalog.load("kvlcc2.ship_data")
vmm = catalog.load("vmm_abkowitz")
exclude_parameters = catalog.load("params:kvlcc2.motion_regression.exclude_parameters")

In [ ]:
def fit_data(data):
    
    mask = (data['u']==0)
    data.loc[mask,'u'] = 0.01
    
    mask = (data['v']==0)
    data.loc[mask,'v'] = 0.01
    
    data = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)
    
    regression, derivatives = fit_motions(data=data, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)
    model = regression.create_model(control_keys=['delta','thrust'])
    
    df_sim = simulate_euler(data=data, model=model, ek=ek)
    
    return regression, model, df_sim
    

In [ ]:
regressions = {}
models = {}
simulations = {}

for dataset_name, data in dataframes.items():
    
    regression, model, df_sim = fit_data(data=data)
    regressions[dataset_name] = regression
    models[dataset_name] = model
    simulations[dataset_name] = df_sim
    

In [ ]:
dataframes_ = {
    'model test':data,
}
dataframes_.update(simulations)
dataframes_.pop('raw')
plot(dataframes_, keys=["delta", "psi", "u", "v", "r", "u1d","v1d","r1d"]);

In [ ]:
simulations['smooth'].isnull().any()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,15)
track_plots(dataframes_, lpp=ship_data['L'], beam=ship_data['B'], ax=ax)

In [ ]:
%reload_kedro

#id = "MARIN_FREE_KVLCC2_tc_35_m"
id = "MARIN_FREE_KVLCC2_zz_20_m"
#id = "MARIN_FREE_KVLCC2_tc_35_m"

data = catalog.load(f"kvlcc2.{id}.data")

dataframes_ = {'initial': catalog.load(f"kvlcc2.initial.{id}.data_ek_smooth"),
               'updated': catalog.load(f"kvlcc2.updated.{id}.data_ek_smooth")}

parameters_initial = catalog.load(f"kvlcc2.initial.vmm_martin.joined.derivatives")
parameters_updated = catalog.load(f"kvlcc2.updated.vmm_martin.joined.derivatives")

plot(dataframes_);

In [ ]:
df_parameters = pd.DataFrame()
df_parameters['initial'] = parameters_initial['regressed']
df_parameters['updated'] = parameters_updated['regressed']
df_parameters.plot.bar()